In [1]:
from databroker.v2 import temp
from bluesky import RunEngine, Msg
from bluesky.plans import count
import bluesky.plan_stubs as bps

import bluesky.preprocessors as bpp

from secop_ophyd.SECoPDevices import SECoP_Node_Device, SECoPReadableDevice, SECoPMoveableDevice, SECoP_CMD_Device
import asyncio
import random



# Create a run engine and a temporary file backed database. Send all the documents from the RE into that database
RE = RunEngine({})
db = temp()
RE.subscribe(db.v1.insert)

from bluesky.plans import scan
from ophyd.sim import det,det1, motor

#Example of adding metadata to RE environment

investigation_id = "kmnk2n34"

RE.md["investigation_id"] = investigation_id

robo_dev = await SECoP_Node_Device.create('localhost','10770',RE.loop)

# now do something
RE(scan([det],motor,1,2,10),reason="for a demo")

storage:SECoPReadableDevice= robo_dev.storage
sample:SECoPMoveableDevice= robo_dev.sample
loadshort:SECoP_CMD_Device = storage.load_short_dev
measure:SECoP_CMD_Device = sample.measure_dev


sample_changer.HZB ready


In [2]:
await robo_dev.read_configuration()



reading = await storage.value.read()


reading = await storage.value.read()
print(reading)


loadshort:SECoP_CMD_Device = storage.load_short_dev

for samplepos in range(1,13):
    await loadshort.samplepos_arg.set(samplepos)
    await loadshort.substance_arg.set(random.randint(0,6))

    await loadshort.load_short_x.execute()

    await storage.wait_for_IDLE()
    print(samplepos)
        
        

{'sample_changer-HZB-storage-value': {'value': [], 'timestamp': 1693837737.9337869}}
1
2
3
4
5
6
7
8
9
10
11
12


In [ ]:
await robo_dev.storage.read()
#await robo_dev.storage.read_configuration()

RE(count([storage],delay=2,num=5))


try to reconnect to localhost:10770
can not connect to localhost:10770, [Errno 111] Connection refused


('a44c6a7f-f613-48b0-a7e4-531c060aeda3',)

In [4]:


def measure(detectors,sample,sample_num):
        

    
    reading = yield from bps.read(sample)
  
    
    curr_sample = reading[sample.value.name]['value']
    
    # holding wrong sample -->  put it back into storage
    if curr_sample != 0 and curr_sample  != sample_num :
        yield from bps.mv(sample,0)
    
    # gripper empty --> grab correct sample
    if curr_sample == 0:
        yield from bps.mv(sample,i)
    
    # Do actual measurement
    @bpp.stage_decorator(detectors)
    @bpp.run_decorator()
    def inner_meas(detectors,sample):

        complete_status = yield from bps.complete(sample.measure_dev, wait=False) #This message doesn't exist yet
    
        # While the device is still executing, read from the detectors in the detectors list
        while not complete_status.done:

            yield Msg('checkpoint') # allows us to pause the run 
            yield from bps.one_shot(detectors) #triggers and reads everything in the detectors list
            yield Msg('sleep', None, 1)       
    
    uid = yield from inner_meas(detectors,sample)


    # put sample back into storage
    yield from  bps.mv(sample,0)

    return uid




for i in range(1,13):
    #grab sample i and hold in Measurement Pos
    RE(measure([det,det1],sample,i))
    

    
